1. Vogel’s Approximation Method (VAM)

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment

In [ ]:
# Data biaya transportasi (supply ke demand)
cost = np.array([[19, 30, 50, 10], 
                 [70, 30, 40, 60], 
                 [40, 8, 70, 20]])

In [ ]:
# Supply dan Demand
supply = np.array([50, 60, 50])  # Kapasitas tiap pabrik
demand = np.array([30, 20, 70, 40])  # Permintaan tiap gudang

In [ ]:
# Fungsi Vogel's Approximation Method (VAM)
def vogel_approximation(cost, supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(cost)
    
    while np.any(supply > 0) and np.any(demand > 0):
        penalties = []

        # Hitung penalti baris
        for i, row in enumerate(cost):
            valid_values = row[demand > 0]
            if len(valid_values) > 1:
                penalties.append(sorted(valid_values)[:2])
            else:
                penalties.append([valid_values[0], valid_values[0]])

        row_penalties = [abs(p[1] - p[0]) for p in penalties]

        # Hitung penalti kolom
        penalties = []
        for j, col in enumerate(cost.T):
            valid_values = col[supply > 0]
            if len(valid_values) > 1:
                penalties.append(sorted(valid_values)[:2])
            else:
                penalties.append([valid_values[0], valid_values[0]])

        col_penalties = [abs(p[1] - p[0]) for p in penalties]

        # Pilih indeks penalti terbesar
        row_max = max(row_penalties) if row_penalties else -1
        col_max = max(col_penalties) if col_penalties else -1

        if row_max >= col_max:
            i = row_penalties.index(row_max)
            j = np.argmin(cost[i][demand > 0])
        else:
            j = col_penalties.index(col_max)
            i = np.argmin(cost.T[j][supply > 0])

        # Alokasikan nilai minimum antara supply dan demand
        qty = min(supply[i], demand[j])
        allocation[i, j] = qty
        supply[i] -= qty
        demand[j] -= qty

    return allocation

In [ ]:
# Jalankan metode VAM
allocation = vogel_approximation(cost, supply, demand)

In [ ]:
# Hitung total biaya transportasi
total_cost = np.sum(allocation * cost)

In [ ]:
# Hasil
print("Hasil Alokasi Transportasi (VAM):")
print(pd.DataFrame(allocation))
print(f"Total Biaya Transportasi: {total_cost}")

2. MODI (Modified Distribution Method)

In [ ]:
from scipy.optimize import linprog

In [ ]:
# MODI membutuhkan persamaan linear
num_sources, num_destinations = cost.shape
c = cost.flatten()
A_eq = []
b_eq = np.concatenate((supply, demand))

In [ ]:
# Matriks kendala
for i in range(num_sources):
    row = np.zeros(cost.shape).flatten()
    row[i * num_destinations:(i + 1) * num_destinations] = 1
    A_eq.append(row)

for j in range(num_destinations):
    col = np.zeros(cost.shape).flatten()
    col[j::num_destinations] = 1
    A_eq.append(col)

A_eq = np.array(A_eq)

In [ ]:
# Batasan variabel (semua harus >= 0)
bounds = [(0, None)] * len(c)

In [ ]:
# Menjalankan MODI menggunakan linprog
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

In [ ]:
# Bentuk solusi ke matriks
optimal_allocation = result.x.reshape(cost.shape)

In [ ]:
# Hasil
print("\nHasil Alokasi Optimal (MODI):")
print(pd.DataFrame(optimal_allocation))
print(f"Total Biaya Transportasi Optimal: {result.fun}")

3. Least Unit Cost

In [ ]:
# Data biaya per unit, supply, dan demand
cost = np.array([[2, 3, 1], [5, 4, 8], [5, 6, 8]])
supply = np.array([50, 60, 50])
demand = np.array([30, 50, 80])

In [ ]:
# Least Unit Cost Method
allocation = np.zeros_like(cost)

In [ ]:
while np.any(supply > 0) and np.any(demand > 0):
    min_cost_idx = np.unravel_index(np.argmin(cost, where=(supply[:, None] > 0) & (demand > 0), initial=np.inf), cost.shape)
    i, j = min_cost_idx
    qty = min(supply[i], demand[j])
    allocation[i, j] = qty
    supply[i] -= qty
    demand[j] -= qty

In [ ]:
# Total biaya
total_cost = np.sum(allocation * cost)

In [ ]:
print("Hasil Alokasi Least Unit Cost:")
print(allocation)
print(f"Total Biaya Transportasi: {total_cost}")